
# Storm tracking - Cyclone Phase Space (CPS) diagram

*Author : Frédéric FERRY - Météo-France / Ecole Nationale de la Météorologie (June 2023)*

Concepts illustrated here :
- Storm tracking from ERA5 data in netcdf format (geopotential height / mean sea level pressure) and a specified tracking file in .txt format
- Computation of the cyclone phase space parameters along the track (thermal/thickness asymmetry and warm core/cold core trough thermal wind relation)
- Creation of the CPS diagram like in http://moe.met.fsu.edu/cyclonephase/

ERA5 data in netcdf format can be downloaded here:
- https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=form
- https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-pressure-levels?tab=form

In [ ]:
import os

import xarray as xr
import netCDF4

import math
import numpy as np

from cartopy import config
from cartopy.util import add_cyclic_point
import cartopy.feature as cfeature
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
import matplotlib.collections as collections

import IPython.display as IPdisplay, matplotlib.font_manager as fm
from PIL import Image
import glob
from tqdm import tqdm

from sklearn.linear_model import LinearRegression
from scipy.ndimage import maximum_filter, minimum_filter

In [ ]:
storm='Zorbas'
#storm='Leslie2'

if not os.path.exists('./anim/'+storm):
    os.mkdir('./anim/'+storm)

if not os.path.exists('./figs/'+storm):
    os.mkdir('./figs/'+storm)
    
dir_anim='./anim/'+storm+'/'

In [ ]:
def plot_maxmin_points(data, extrema, nsize, symbol, color='k',
                       plotValue=True, transform=None):

    if (extrema == 'max'):
        data_ext = maximum_filter(data, nsize, mode='nearest')
    elif (extrema == 'min'):
        data_ext = minimum_filter(data, nsize, mode='nearest')
    else:
        raise ValueError('Value for hilo must be either max or min')

    mxy, mxx = np.where(data_ext == data)

    for i in range(len(mxy)):
        ax.text(data.longitude[mxx[i]].values, data.latitude[mxy[i]].values, symbol, color=color, size=12,
                clip_on=True, horizontalalignment='center', verticalalignment='center',
                transform=transform)
        ax.text(data.longitude[mxx[i]].values, data.latitude[mxy[i]].values,
                '\n' + str(int(data[mxy[i], mxx[i]])),
                color=color, size=10, clip_on=True, fontweight='bold',
                horizontalalignment='center', verticalalignment='top', transform=transform)

In [ ]:
def make_animation(gif_filepath):
    from PIL import Image
    import os
    from IPython.display import Image as IPImage
    from IPython.display import display
    import time
    
    image_folder = './anim/'+storm+'/' # répertoire contenant les fichiers PNG
    output_file = gif_filepath # nom du fichier de sortie
    animation_speed = 0.9 # vitesse de l'animation en secondes
    
    # Liste tous les fichiers PNG dans le répertoire image_folder
    files = sorted(os.listdir(image_folder))
    image_files = [f for f in files if f.endswith('.png')]
    
    # Ouvre chaque fichier PNG et ajoute l'image à une liste
    images = []
    for filename in image_files:
        img = Image.open(os.path.join(image_folder, filename))
        images.append(img)
    
    # Crée l'animation GIF
    images[0].save(output_file, save_all=True, append_images=images[1:], duration=int(animation_speed*1000), loop=0)
    # Affiche l'animation GIF dans Jupyter
    with open(output_file,'rb') as f:
        display(IPImage(data=f.read(), format='png'))
    # Efface les fichiers PNG
    for filename in image_files:
        os.remove(image_folder+filename)

In [ ]:
def haversine(lon1, lat1, lon2, lat2):
    # convert decimal degrees to radians 
    lon1 = np.deg2rad(lon1)
    lon2 = np.deg2rad(lon2)
    lat1 = np.deg2rad(lat1)
    lat2 = np.deg2rad(lat2)

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    r = 6371
    return c * r

In [ ]:
def get_bearing(lat1, lon1, lat2, lon2):
    dLon = (lon2 - lon1)
    x = math.cos(math.radians(lat2)) * math.sin(math.radians(dLon))
    y = math.cos(math.radians(lat1)) * math.sin(math.radians(lat2)) - math.sin(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.cos(math.radians(dLon))
    brng = np.arctan2(x,y)
    brng = np.degrees(brng)
    brng = (brng +360) % 360
    return brng

# Check tracking file

In [ ]:
liste_time,liste_lon,liste_lat,liste_pres=np.loadtxt('./txt/'+storm+'.txt',
        skiprows=0, dtype='U13,float,float,int',unpack=True)
print(liste_time)

In [ ]:
interval=int(input("Enter the desired time interval : "))

liste_time=liste_time[::interval]
liste_lon=liste_lon[::interval]
liste_lat=liste_lat[::interval]
liste_pres=liste_pres[::interval]

print(liste_time)
print(liste_time.shape)

date1=str(liste_time[0])
date2=str(liste_time[-1])

In [ ]:
def plot_background(ax):
    ax.coastlines()
    ax.gridlines()
    ax.set_xticks(np.linspace(-180, 180, 37), crs=ccrs.PlateCarree())
    ax.set_yticks(np.linspace(-90, 90, 37), crs=ccrs.PlateCarree())
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)
    return(ax)

projection=ccrs.PlateCarree()

fig = plt.figure(figsize=(20., 6.))
ax = fig.add_subplot(121, projection=projection)
ax.set_title(storm+' storm track',loc='center',fontsize=14)
ax.stock_img()
plot_background(ax)
ax.set_extent([5, 35, 30, 45])

ax.plot(liste_lon,liste_lat, color='red', transform=ccrs.PlateCarree())
ax.scatter(liste_lon,liste_lat, color='red', transform=ccrs.PlateCarree())
for i in range(len(liste_time)):
    ax.text(liste_lon[i], liste_lat[i], liste_pres[i],verticalalignment='top', horizontalalignment='center',
            transform=ccrs.PlateCarree())

ax = fig.add_subplot(122)
ax.set_title(storm+' MSP evolution',loc='center',fontsize=14)
ax.plot(liste_time,liste_pres, label=storm)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Mean sea level pressure (hPa)', fontsize=12)
locs, labels = plt.xticks()
plt.setp(labels, rotation=90)
plt.grid()
ax.tick_params(axis='x', labelsize=7)

plt.show()

figname='./figs/'+storm+'/MSLP_evolution'
fig.savefig(figname+'.png')

# Check available ERA5 data

In [ ]:
f1    = xr.open_dataset("./data/"+storm+"/msl.nc")
f2    = xr.open_dataset("./data/"+storm+"/z.nc")
print(f1)
print(f2)

In [ ]:
interval=int(input("Enter the desired time interval : "))

# Open and plot data

In [ ]:
if storm=='Zorbas':
    latS=30
    latN=45
    lonW=5
    lonE=35
else:
    latS=20
    latN=45
    lonW=-60
    lonE=0

In [ ]:
f1    = xr.open_dataset("./data/"+storm+"/msl.nc").sel(
    time=slice(date1,date2)).sel(
    latitude=slice(latN,latS)).sel(
    longitude=slice(lonW,lonE))

mslp = f1['msl'][::interval]/100
lat  = mslp.latitude.values
time  = mslp.time.values
lon  = mslp.longitude.values

print(mslp)

In [ ]:
f2    = xr.open_dataset("./data/"+storm+"/z.nc").sel(
    time=slice(date1,date2)).sel(
    latitude=slice(latN,latS)).sel(
    longitude=slice(lonW,lonE))

lev = f2.level.values

z = f2['z'][::interval]/9.81
print(z)

In [ ]:
print(lev)
lower_layer = int(input('Enter 1 for 900-600 hPa lower layer and 2 for 925-700 hPa lower layer : ')) 
if (lower_layer==1):
    listlev1=[900, 800, 700, 600]
if (lower_layer==2):
    listlev1=[925, 875, 825, 775, 700]
higher_layer = int(input('Enter 1 for 600-300 hPa higher layer and 2 for 700-400 hPa higher layer : ')) 
if (higher_layer==1):
    listlev2=[600, 500, 400, 300]
if (higher_layer==2):
    listlev2=[700, 600, 500, 400]

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) </b>According to the hypsometric equation (derived from the hydrostatic balance) to which atmospheric parameter is the tickness of an atmospheric layer equivalent ?</p>
    
https://en.wikipedia.org/wiki/Hypsometric_equation
    
</div>

<div class="alert alert-success">
<b>Answer </b>
</div>

<div class="alert alert-danger">
<p><b>Compute the lower layer thickness from the z DataArray and using xarray's sel method : thickness = Zhigh-Zlow.</b></p>
</div>

In [ ]:
print(listlev1)
print(z)
thickness=
print(thickness)
print(np.min(thickness))
print(np.max(thickness))

In [ ]:
mslp_levels = np.arange(900,1072,2)
#dz_levels = np.arange(3000,3525,25)
dz_levels=np.linspace(np.min(thickness),np.max(thickness),20)

projection=ccrs.PlateCarree()
bounds = [(lonW, lonE, latS, latN)]
#coast = cfeature.NaturalEarthFeature(category='physical', scale='10m', facecolor='none', name='coastline')

for i in tqdm(range(len(time))):
    #print(liste_time[i])
    fig = plt.figure(figsize=(17., 12.))
    #fig.suptitle('Storm '+storm+' - Mean Sea level Pressure and tracking',fontsize=16)
    ax = fig.add_subplot(1, 1, 1, projection=projection)
    ax.set_title('Storm '+storm+' - MSLP tracking and low level thickness',loc='left',fontsize=14)
    ax.set_title(liste_time[i],loc='right',fontsize=14)
    ax.coastlines()
    #ax.add_feature(coast, edgecolor='gray')
    ax.set_xticks(np.linspace(-180, 180, 37), crs=ccrs.PlateCarree())
    ax.set_yticks(np.linspace(-90, 90, 19), crs=ccrs.PlateCarree())
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)
    ax.set_extent(*bounds, crs=ccrs.PlateCarree())
    
    # MSLP in contours and min max
    c1 = ax.contour(lon, lat, mslp[i,:,:], levels=mslp_levels, 
                    colors="black", linewidths=1, transform=ccrs.PlateCarree())
    ax.clabel(c1,fmt='%4.1i',fontsize=10)
    ax.scatter(liste_lon[i],liste_lat[i], c='green', transform=ccrs.PlateCarree())
    ax.plot(liste_lon[0:i+1],liste_lat[0:i+1], c='red', marker='+', transform=ccrs.PlateCarree())
    plot_maxmin_points(mslp[i,:,:], 'min', 25,
                       symbol='L', color='b', transform=ccrs.PlateCarree())
    cf = ax.contourf(lon, lat, thickness[i,:,:], levels=dz_levels, 
                     cmap='jet', transform=ccrs.PlateCarree())
    cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.1)
    cb.set_label('m')
    #plt.show()
    figname='./anim/'+storm+'/MSL_thickness_tracking_'+liste_time[i]
    fig.savefig(figname+'.png',bbox_inches='tight')
    plt.close()

In [ ]:
gif_filepath = './anim/'+storm+'/MSL_thickness_tracking_.gif'
make_animation(gif_filepath)

# Choosing a radius for the computation of the CPS parameters

In [ ]:
max_dist = 500      # max distance in km

for i in tqdm(range(len(time))):
    #print(liste_time[i])
    data_slp = mslp[i,:,:]
    data_thickness = thickness[i,:,:]
        
    # Center coordinates
    clat = liste_lat[i]
    clon = liste_lon[i] 
        
    # Calculate distance between center and all other lat/lon pairs
    distance = haversine(lon[:,np.newaxis], lat, clon, clat)
    distance2=np.transpose(distance)
    # Mask distance array where distance > max_dist
    distance_m = np.ma.masked_greater(distance2, max_dist)
    # Mask the data array based on the distance mask
    data_slp_m = np.ma.masked_where(distance2 > max_dist, data_slp)
    data_thickness_m = np.ma.masked_where(distance2 > max_dist, data_thickness)

    fig = plt.figure(figsize=(17., 12.))
    ax = fig.add_subplot(1, 1, 1, projection=projection)
    ax.set_title('Storm '+storm+' - MSLP tracking and low level thickness',loc='left',fontsize=14)
    ax.set_title(liste_time[i],loc='right',fontsize=14)
    ax.coastlines()
    #ax.add_feature(coast, edgecolor='gray')
    ax.set_xticks(np.linspace(-180, 180, 37), crs=ccrs.PlateCarree())
    ax.set_yticks(np.linspace(-90, 90, 19), crs=ccrs.PlateCarree())
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)
    ax.set_extent(*bounds, crs=ccrs.PlateCarree())
    # MSLP in contours and min max
    c1 = ax.contour(lon, lat, data_slp_m, levels=mslp_levels,
                    colors="black", linewidths=1, transform=ccrs.PlateCarree())
    ax.clabel(c1,fmt='%4.1i',fontsize=10)
    ax.scatter(liste_lon[i],liste_lat[i], c='green', transform=ccrs.PlateCarree())
    ax.plot(liste_lon[0:i+1],liste_lat[0:i+1], c='red', marker='+', transform=ccrs.PlateCarree())
    cf = ax.contourf(lon, lat, data_thickness_m, levels=dz_levels, 
                     cmap='jet', transform=ccrs.PlateCarree())
    cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.1)
    cb.set_label('m')
    #plt.show()
    figname='./anim/'+storm+'/MSL_thickness_tracking_zoom_'+liste_time[i]
    fig.savefig(figname+'.png',bbox_inches='tight')
    plt.close()

In [ ]:
gif_filepath = './anim/'+storm+'/MSL_thickness_tracking_zoom_.gif'
make_animation(gif_filepath)

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) </b>Is the default 500km radius suitable to isolate medicane Zorbas from its environment ?</p>
</div>

<div class="alert alert-success">
<b>Answer </b>
</div>

In [ ]:
max_dist = int(input('Enter the value of the maximum radius (in km) to compute the CPS diagrams : ')) 

# Thermal asymmetry parameter B : storm-motion-relative low level thickness gradient across the cyclone

$$B=h*(\overline{Z_{600}-Z_{900}})_R-\overline{Z_{600}-Z_{900}})_L$$

h = hemisphere (1 = NH, -1 = SH).

$$(\overline{Z_{600}-Z_{900}})_R$$ : mean 900-600hPa thickness in semicircle right of motion

$$(\overline{Z_{600}-Z_{900}})_L$$ : mean 900-600hPa thickness in semicircle left of motion

In [ ]:
print('Available dates : ', liste_time[0:-1])

date = input('Enter a date (YYYY-MM-DDTXX) : ')
data = thickness.sel(time=date)
data_slp = mslp.sel(time=date)
print(data)

idate = list(liste_time).index(date)
print('index of date : ', idate)

In [ ]:
clat = liste_lat[idate]
clon = liste_lon[idate]
ang=get_bearing(clat, clon, liste_lat[idate+1], liste_lon[idate+1])

In [ ]:
qq_ang_all=np.zeros([len(lon),len(lat)],dtype='f')
for x in range(0,len(lon)):
    for y in range(0,len(lat)):
        qq_ang_all[x,y]=get_bearing(clat, clon, lat[y], lon[x])     
qq_ang_all=np.transpose(qq_ang_all)

In [ ]:
Zl=np.zeros([len(lat),len(lon)],dtype='f')
Zr=np.zeros([len(lat),len(lon)],dtype='f')

for jlat in range(0,len(lat)-1):
    for jlon in range(0,len(lon)-1):
        #set values along great circle line to missing (neither left nor right)
        if qq_ang_all[jlat,jlon] == ang:
            Zl[jlat,jlon] = 0
            Zr[jlat,jlon] = 0
        #for storm travel angles in quadrants 1 and 2 (NE and SE)
        elif (ang >= 0 and ang < 180):
            if (qq_ang_all[jlat,jlon] > ang and qq_ang_all[jlat,jlon]  < ang+180):
                Zl[jlat,jlon] = 0
                Zr[jlat,jlon] = data[jlat,jlon]
            else:
                Zr[jlat,jlon] = 0
                Zl[jlat,jlon] = data[jlat,jlon]
        #for storm travel angles in quadrants 3 and 4 (NW and SW)
        elif (ang >= 180 and ang < 360):
            if (qq_ang_all[jlat,jlon] > ang-180 and qq_ang_all[jlat,jlon] < ang):
                Zr[jlat,jlon] = 0
                Zl[jlat,jlon] = data[jlat,jlon]
            else:
                Zl[jlat,jlon] = 0
                Zr[jlat,jlon] = data[jlat,jlon]

In [ ]:
# Calculate distance between center and all other lat/lon pairs
distance = haversine(lon[:,np.newaxis], lat, clon, clat)
distance2=np.transpose(distance)
# Mask distance array where distance > max_dist
distance_m = np.ma.masked_greater(distance2, max_dist)
# Mask the data array based on the distance mask
data_slp_m = np.ma.masked_where(distance2 > max_dist, data_slp)
Zr_m = np.ma.masked_where(distance2 > max_dist, Zr)
Zl_m = np.ma.masked_where(distance2 > max_dist, Zl)

Zr_m[Zr_m == 0] = np.nan
Zl_m[Zl_m == 0] = np.nan

In [ ]:
mean_calc = int(input('Enter 1 for classic average and 2 for weighted mean average : ')) 

if (mean_calc==1):
    # cyclone thermal symmetry parameter using classic average
    Zr_mean=np.nanmean(Zr_m)
    Zl_mean=np.nanmean(Zl_m)
    B=Zr_mean-Zl_mean
    print('Right quadrant thickness : ', Zr_mean)
    print('Left quadrant thickness : ', Zl_mean)

if (mean_calc==2):
    # cyclone thermal symmetry parameter using weighted average
    Zr_m = xr.DataArray(Zr_m)
    Zl_m = xr.DataArray(Zl_m)
    weights = xr.DataArray(np.cos(np.deg2rad(Zr_m.shape[0])))
    Zr_weighted = Zr_m.weighted(weights)
    Zl_weighted = Zl_m.weighted(weights)
    Zr_weighted_mean=Zr_weighted.mean()
    Zl_weighted_mean=Zl_weighted.mean()
    B=Zr_weighted_mean-Zl_weighted_mean
    print(Zr_weighted_mean)
    print(Zl_weighted_mean)

if clat<0:
    B=-B

In [ ]:
levels=np.arange(0,370,10)

fig = plt.figure(figsize=(17., 12.))
ax = fig.add_subplot(1, 1, 1, projection=projection)
fig.suptitle('Storm '+storm+' - MSLP, low level thickness and right/left quadrants',fontsize=16)
ax.set_title(liste_time[idate]+' : B = '+str(int(B)),fontsize=14)
ax.coastlines()
#ax.add_feature(coast, edgecolor='gray')
ax.set_xticks(np.linspace(-180, 180, 37), crs=ccrs.PlateCarree())
ax.set_yticks(np.linspace(-90, 90, 19), crs=ccrs.PlateCarree())
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax.xaxis.set_major_formatter(lon_formatter)
ax.yaxis.set_major_formatter(lat_formatter)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
 
c0 = ax.contour(lon, lat, data_slp_m, mslp_levels,
                colors="black", linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c0,fmt='%4.1i',fontsize=10)
cf1 = ax.contourf(lon, lat, Zr_m, dz_levels,
                 cmap='jet', transform=ccrs.PlateCarree())
cf2 = ax.contourf(lon, lat, Zl_m, dz_levels,
                 cmap='jet', transform=ccrs.PlateCarree())
cb = fig.colorbar(cf1, orientation='horizontal', aspect=65, shrink=0.5, pad=0.1)
cb.set_label('m')
c3 = ax.contour(lon, lat, qq_ang_all, levels,
                colors="grey", linewidths=0.5, alpha=0.8, transform=ccrs.PlateCarree())
ax.clabel(c3,fmt='%4.1i',fontsize=10)

plt.show()

figname='./figs/'+storm+'/Thermal_symmetry'
fig.savefig(figname+'.png',bbox_inches='tight')

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) </b>According to the definition of B parameter, do B values close to zero charaterise a symmetric or asymmetric thermal structure ?</p>
<p><b>2) </b>According to the definition of B parameter, do strong B values charaterise a symmetric or asymmetric thermal structure ?</p></div>

<div class="alert alert-success">
<b>Answer </b>
</div>

In [ ]:
B_series=[]

for i in tqdm(range(len(time)-1)):
    #print(liste_time[i])
    data = thickness[i,:,:]
    data_slp = mslp[i,:,:]

    # Center coordinates
    clat = liste_lat[i]
    clon = liste_lon[i]

    # Calculate bearing between center and all other lat/lon pairs
    qq_ang_all=np.zeros([len(lon),len(lat)],dtype='f')
    for x in range(0,len(lon)):
        for y in range(0,len(lat)):
            qq_ang_all[x,y]=get_bearing(clat, clon, lat[y], lon[x])     
    qq_ang_all=np.transpose(qq_ang_all)

    # Angle of travel of the center
    ang=get_bearing(clat, clon, liste_lat[i+1], liste_lon[i+1])

    # Define right/left cadrans
    Zl=np.zeros([len(lat),len(lon)],dtype='f')
    Zr=np.zeros([len(lat),len(lon)],dtype='f')

    for jlat in range(0,len(lat)-1):
        for jlon in range(0,len(lon)-1):
            #set values along great circle line to missing (neither left nor right)
            if qq_ang_all[jlat,jlon] == ang:
                Zl[jlat,jlon] = 0
                Zr[jlat,jlon] = 0
            #for storm travel angles in quadrants 1 and 2 (NE and SE)
            elif (ang >= 0 and ang < 180):
                if (qq_ang_all[jlat,jlon] > ang and qq_ang_all[jlat,jlon]  < ang+180):
                    Zl[jlat,jlon] = 0
                    Zr[jlat,jlon] = data[jlat,jlon]
                else:
                    Zr[jlat,jlon] = 0
                    Zl[jlat,jlon] = data[jlat,jlon]
            #for storm travel angles in quadrants 3 and 4 (NW and SW)
            elif (ang >= 180 and ang < 360):
                if (qq_ang_all[jlat,jlon] > ang-180 and qq_ang_all[jlat,jlon] < ang):
                    Zr[jlat,jlon] = 0
                    Zl[jlat,jlon] = data[jlat,jlon]
                else:
                    Zl[jlat,jlon] = 0
                    Zr[jlat,jlon] = data[jlat,jlon]

    # Calculate distance between center and all other lat/lon pairs
    distance = haversine(lon[:,np.newaxis], lat, clon, clat)
    distance2=np.transpose(distance)
    # Mask distance array where distance > max_dist
    distance_m = np.ma.masked_greater(distance2, max_dist)
    # Mask the data array based on the distance mask
    data_slp_m = np.ma.masked_where(distance2 > max_dist, data_slp)
    Zr_m = np.ma.masked_where(distance2 > max_dist, Zr)
    Zl_m = np.ma.masked_where(distance2 > max_dist, Zl)
    
    Zr_m[Zr_m == 0] = np.nan
    Zl_m[Zl_m == 0] = np.nan

    # cyclone thermal symmetry parameter using weighted average
    #Zr_mean=np.nanmean(Zr_m)
    #Zl_mean=np.nanmean(Zl_m)
    #B=Zr_mean-Zl_mean

    Zr_m = xr.DataArray(Zr_m)
    Zl_m = xr.DataArray(Zl_m)
    weights = xr.DataArray(np.cos(np.deg2rad(Zr_m.shape[0])))
    Zr_weighted = Zr_m.weighted(weights)
    Zl_weighted = Zl_m.weighted(weights)
    Zr_weighted_mean=Zr_weighted.mean()
    Zl_weighted_mean=Zl_weighted.mean()
    B=int(Zr_weighted_mean-Zl_weighted_mean)

    if clat<0:
        B=-B
    #print('B = ',B)
    
    B_series.append(B)

    fig = plt.figure(figsize=(17., 12.))
    ax = fig.add_subplot(1, 1, 1, projection=projection)
    fig.suptitle('Storm '+storm+' - MSLP, low level thickness and right/left quadrants',fontsize=16)
    ax.set_title(liste_time[i]+' : B = '+str(int(B)),fontsize=14)
    ax.coastlines()
    #ax.add_feature(coast, edgecolor='gray')
    ax.set_xticks(np.linspace(-180, 180, 37), crs=ccrs.PlateCarree())
    ax.set_yticks(np.linspace(-90, 90, 19), crs=ccrs.PlateCarree())
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)
    ax.set_extent(*bounds, crs=ccrs.PlateCarree())

    c0 = ax.contour(lon, lat, data_slp_m, mslp_levels,
                    colors="black", linewidths=1, transform=ccrs.PlateCarree())
    ax.clabel(c0,fmt='%4.1i',fontsize=10)
    cf1 = ax.contourf(lon, lat, Zr_m, dz_levels,
                     cmap='jet', transform=ccrs.PlateCarree())
    cf2 = ax.contourf(lon, lat, Zl_m, dz_levels,
                     cmap='jet', transform=ccrs.PlateCarree())
    cb = fig.colorbar(cf1, orientation='horizontal', aspect=65, shrink=0.5, pad=0.1)
    cb.set_label('m')
    c3 = ax.contour(lon, lat, qq_ang_all, levels,
                    colors="grey", linewidths=0.5, alpha=0.8, transform=ccrs.PlateCarree())
    ax.clabel(c3,fmt='%4.1i',fontsize=10)

    #plt.show()
    figname='./anim/'+storm+'/Thermal_symmetry_'+liste_time[i]
    fig.savefig(figname+'.png',bbox_inches='tight')
    plt.close()

In [ ]:
gif_filepath = './anim/'+storm+'/Thermal_symmetry.gif'
make_animation(gif_filepath)

In [ ]:
print(B_series)

fig = plt.figure(figsize=(15., 8.))
ax = fig.add_subplot(111)
ax.set_title('Storm '+storm+' - low level thermal asymmetry parameter (B)'
             ,loc='left',fontsize=14)
ax.set_title(liste_time[0]+'-'+liste_time[-2],loc='right',fontsize=14)
ax.plot(time[0:-1], B_series, label='B')
plt.axhline(y=10, color='r')

figname='./figs/'+storm+'/Thermal_symmetry_timeseries'
fig.savefig(figname+'.png',bbox_inches='tight')

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) </b>Discuss the evolution of the low level thermal asymmetry of the system during its life cycle.</p>
</div>

<div class="alert alert-success">
<b>Answer </b>
</div>

# Warm core/ Cold core parameter (using the thermal wind relation)

Approximation of the height gradient on an isobaric surface (i.e. magnitude of the geostrophic wind) :

$$\Delta{Z}=Z_{max}-Z_{min}$$

Magnitude of the cyclone thermal wind measured as the vertical profile of cyclone geopotential height gradient (cyclone strength) :

$$-V_T^L=\frac{\partial{\Delta Z}}{\partial{ln(P)}}_{900}^{600}$$

$$-V_T^U=\frac{\partial{\Delta Z}}{\partial{ln(P)}}_{600}^{300}$$

In [ ]:
print('Available dates : ', liste_time[0:-1])

date = input('Enter a date (YYYY-MM-DDTXX) : ')
data = thickness.sel(time=date)
data_slp = mslp.sel(time=date)
print(data)

idate = list(liste_time).index(date)
print('index of date : ', idate)

In [ ]:
clat = liste_lat[idate]
clon = liste_lon[idate]

deltaZ1=[]
deltaZ2=[]

#low layer loop
for plev in listlev1:

    ilev = list(lev).index(plev)
    data_z1 = z[idate,ilev,:,:]

    # Calculate distance between center and all other lat/lon pairs
    distance = haversine(lon[:,np.newaxis], lat, clon, clat)
    distance2=np.transpose(distance)
    # Mask distance array where distance > max_dist
    distance_m = np.ma.masked_greater(distance2, max_dist)
    # Mask the data array based on the distance mask
    data_z1_m = np.ma.masked_where(distance2 > max_dist, data_z1)
    zmin=np.min(data_z1_m)
    zmax=np.max(data_z1_m)
    delta_z1=zmax-zmin
    deltaZ1.append(int(delta_z1))

#high layer loop
for plev in listlev2:

    ilev = list(lev).index(plev)
    data_z2 = z[idate,ilev,:,:]

    # Calculate distance between center and all other lat/lon pairs
    distance = haversine(lon[:,np.newaxis], lat, clon, clat)
    distance2=np.transpose(distance)
    # Mask distance array where distance > max_dist
    distance_m = np.ma.masked_greater(distance2, max_dist)
    # Mask the data array based on the distance mask
    data_z2_m = np.ma.masked_where(distance2 > max_dist, data_z2)
    zmin=np.min(data_z2_m)
    zmax=np.max(data_z2_m)
    delta_z2=zmax-zmin
    deltaZ2.append(int(delta_z2))

In [ ]:
print(zmax)
print(zmin)

fig = plt.figure(figsize=(17., 12.))
ax = fig.add_subplot(1, 1, 1, projection=projection)
fig.suptitle('Storm '+storm+' - Geopotential at '+str(plev)+ 'hPa',fontsize=16)
ax.set_title(liste_time[idate]+ ' : zmax-zmin = '+str(int(delta_z2)),fontsize=14)
ax.coastlines()
#ax.add_feature(coast, edgecolor='gray')
ax.set_xticks(np.linspace(-180, 180, 37), crs=ccrs.PlateCarree())
ax.set_yticks(np.linspace(-90, 90, 19), crs=ccrs.PlateCarree())
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax.xaxis.set_major_formatter(lon_formatter)
ax.yaxis.set_major_formatter(lat_formatter)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
 
cf = ax.contourf(lon, lat, data_z2_m,
                     cmap='jet', transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.1)
cb.set_label('m')

plt.show()

figname='./figs/'+storm+'/deltaz_map'
fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
print("Lower layer vertical levels : ", listlev1)
print("Lower layer deltaZ : ", deltaZ1)
print("Higher layer vertical levels : ", listlev2)
print("Higher layer deltaZ : ", deltaZ2)

listlev11=[n * 100 for n in listlev1]
lnP1=np.log(listlev11)

listlev22=[n * 100 for n in listlev2]
lnP2=np.log(listlev22)

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) </b>According to the thermal wind relation, if the slope of an isobaric surface increases with height within a layer above the system center, is it the sign of the presence of a cold core or warm core system ?</p>
<p><b>2) </b>What is the sign of -$V_T$ parameter for a cold (resp. warm) core system ?</p>
</div>

<div class="alert alert-success">
<b>Answer </b>
</div>

In [ ]:
VTL_series=[]
VTU_series=[]

for i in tqdm(range(len(time)-1)):
    #print(liste_time[i])

    # Center coordinates
    clat = liste_lat[i]
    clon = liste_lon[i]
    deltaZ1=[]
    deltaZ2=[]

    for plev in listlev1:

        ilev = list(lev).index(plev)
        data_z1 = z[i,ilev,:,:]
        # Calculate distance between center and all other lat/lon pairs
        distance = haversine(lon[:,np.newaxis], lat, clon, clat)
        distance2=np.transpose(distance)
        # Mask distance array where distance > max_dist
        distance_m = np.ma.masked_greater(distance2, max_dist)
        # Mask the data array based on the distance mask
        data_z1_m = np.ma.masked_where(distance2 > max_dist, data_z1)
        zmin=np.min(data_z1_m)
        zmax=np.max(data_z1_m)
        delta_z1=zmax-zmin
        deltaZ1.append(int(delta_z1))

    for plev in listlev2:

        ilev = list(lev).index(plev)
        data_z2 = z[i,ilev,:,:]

        # Calculate distance between center and all other lat/lon pairs
        distance = haversine(lon[:,np.newaxis], lat, clon, clat)
        distance2=np.transpose(distance)
        # Mask distance array where distance > max_dist
        distance_m = np.ma.masked_greater(distance2, max_dist)
        # Mask the data array based on the distance mask
        data_z2_m = np.ma.masked_where(distance2 > max_dist, data_z2)
        zmin=np.min(data_z2_m)
        zmax=np.max(data_z2_m)
        delta_z2=zmax-zmin
        deltaZ2.append(int(delta_z2))

    coef1=(deltaZ1[-1]-deltaZ1[0])/(lnP1[-1]-lnP1[0])
    VTL_series.append(int(coef1))

    coef2=(deltaZ2[-1]-deltaZ2[0])/(lnP2[-1]-lnP2[0])
    VTU_series.append(int(coef2))

In [ ]:
print(VTU_series)
print(VTL_series)

fig = plt.figure(figsize=(15., 8.))
ax = fig.add_subplot(111)
ax.set_title('Storm '+storm+' - Low level thermal wind parameter'
             ,loc='left',fontsize=14)
ax.set_title(liste_time[0]+'-'+liste_time[-2],loc='right',fontsize=14)
ax.plot(time[0:-1], VTL_series, label='-$VT_T^L$')

plt.axhline(y=0, color='r')

plt.show()

figname='./figs/'+storm+'/VTL_timeseries'
fig.savefig(figname+'.png',bbox_inches='tight')

fig = plt.figure(figsize=(15., 8.))
ax = fig.add_subplot(111)
ax.set_title('Storm '+storm+' - High level thermal wind parameter'
             ,loc='left',fontsize=14)
ax.set_title(liste_time[0]+'-'+liste_time[-2],loc='right',fontsize=14)
ax.plot(time[0:-1], VTU_series, label='-$VT_T^U$')

plt.axhline(y=0, color='r')

plt.show()

figname='./figs/'+storm+'/VTU_timeseries'
fig.savefig(figname+'.png',bbox_inches='tight')

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) </b>Discuss the evolution of the low and high level thermal cores of the system during its life cycle.</p>
</div>

<div class="alert alert-success">
<b>Answer </b>
</div>

In [ ]:
time_str=[x for x in range(len(time)-1)]
date_str=[x for x in range(len(time)-1)]
for i in range(len(time)-1):
    time_str[i] = str(time[i])
    date_str[i] = time_str[i][5:13]

################# CPS1 #################

if storm=='Zorbas':
    x1=-200
    x2=300
    y1=-20
    y2=60

else:
    x1=-600
    x2=300
    y1=-20
    y2=125

fig = plt.figure(figsize=(15., 8.))
ax = fig.add_subplot(111)
ax.set_title('Storm '+storm,loc='left',fontsize=14)
ax.set_title(liste_time[0]+'-'+liste_time[-2],loc='right',fontsize=14)
plt.axhline(y=10, color='grey', linewidth=10,zorder=1)
plt.axvline(x=0, color='grey', linewidth=10,zorder=1)
plt.plot(VTL_series, B_series)
plt.scatter(VTL_series,  B_series, c='k',zorder=2)
plt.scatter(VTL_series[0], B_series[0], c='green',zorder=2)
plt.scatter(VTL_series[-1], B_series[-1], c='red',zorder=2)
plt.xlabel("$-V_T^L$ [low level thermal wind]", fontsize = 16)
plt.ylabel("B [low level thickness asymmetry]", fontsize = 16)

plt.text(x1-35,-15,'Symmetric', rotation=90., color='red', fontsize = 14)
plt.text(x1-35, 20,'Asymmetric', rotation=90., color='red', fontsize = 14)
plt.text(-100, -30,'Cold core', rotation=0., color='blue', fontsize = 14)
plt.text(100, -30,'Warm core', rotation=0., color='red', fontsize = 14)
plt.xlim(x1, x2)
plt.ylim(y1, y2)
    
for i,type in enumerate(date_str):
    x = VTL_series[i]
    y = B_series[i]
    plt.text(x, y, type, fontsize=9)
    
xrange = [(-600, 600)]
yrange1 = (-20, 30)
c1 = collections.BrokenBarHCollection(xrange, yrange1, facecolor='blue', alpha=0.2)
ax.add_collection(c1)
xrange = [(-600, 600)]
yrange1 = (10, 130)
c1 = collections.BrokenBarHCollection(xrange, yrange1, facecolor='blue', alpha=0.2)
ax.add_collection(c1)
xrange = [(0, 300)]
yrange1 = (-20, 30)
c1 = collections.BrokenBarHCollection(xrange, yrange1, facecolor='red', alpha=0.2)
ax.add_collection(c1)
xrange = [(0, 300)]
yrange1 = (10, 130)
c1 = collections.BrokenBarHCollection(xrange, yrange1, facecolor='red', alpha=0.2)
ax.add_collection(c1)

plt.show()

figname='./figs/'+storm+'/CPS1'
fig.savefig(figname+'.png',bbox_inches='tight')

################# CPS2 #################

if storm=='Zorbas':
    x1=-300
    x2=300
    y1=-300
    y2=300

else:
    x1=-600
    x2=300
    y1=-600
    y2=300

fig = plt.figure(figsize=(15., 8.))
ax = fig.add_subplot(111)
ax.set_title('Storm '+storm,loc='left',fontsize=14)
ax.set_title(liste_time[0]+'-'+liste_time[-2],loc='right',fontsize=14)
plt.axhline(y=0, color='grey', linewidth=10,zorder=1)
plt.axvline(x=0, color='grey', linewidth=10,zorder=1)
plt.plot(VTL_series, VTU_series)
plt.scatter(VTL_series, VTU_series, c='k',zorder=2)
plt.scatter(VTL_series[0], VTU_series[0], c='green',zorder=2)
plt.scatter(VTL_series[-1], VTU_series[-1], c='red',zorder=2)

plt.xlabel("$-V_T^L$ [Low level thermal wind]", fontsize = 16)
plt.ylabel("$-V_T^U$ [High level thermal wind]", fontsize = 16)
plt.text(-200, y1+20,'Deep cold core', rotation=0., color='blue', fontsize = 14)
plt.text(90, 270,'Deep warm core', rotation=0., color='red', fontsize = 14)
plt.text(90, y1+20,'Shallow warm core', rotation=0., color='orange', fontsize = 14)

plt.xlim(x1, x2)
plt.ylim(y1, y2)

for i,type in enumerate(date_str):
    x = VTL_series[i]
    y = VTU_series[i]
    plt.text(x, y, type, fontsize=9)

xrange = [(0, 300)]
yrange1 = (0, 300)
c1 = collections.BrokenBarHCollection(xrange, yrange1, facecolor='r', alpha=0.2)
ax.add_collection(c1)
xrange = [(-600, 600)]
yrange1 = (-600, 600)
c1 = collections.BrokenBarHCollection(xrange, yrange1, facecolor='b', alpha=0.2)
ax.add_collection(c1)
xrange = [(-600, 600)]
yrange1 = (0, 300)
c1 = collections.BrokenBarHCollection(xrange, yrange1, facecolor='orange', alpha=0.2)
ax.add_collection(c1)
xrange = [(0, 300)]
yrange1 = (-600, 600)
c1 = collections.BrokenBarHCollection(xrange, yrange1, facecolor='orange', alpha=0.2)
ax.add_collection(c1)

plt.show()

figname='./figs/'+storm+'/CPS2'
fig.savefig(figname+'.png',bbox_inches='tight')

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) </b>According to the CPS diagram, when does the medicane acquire a symmetric deep warm core structure  ?</p>
</div>

<div class="alert alert-success">
<b>Answer </b>
</div>

# Extra task : CPS diagrams for Huricane Leslie over the Atlantic

<div class="alert alert-danger">
<p><b>1) Edit the Leslie.txt file created in the first notebook (extra task) to create a Leslie2.txt file with only 6-hourly values from 2018-10-10 00UTC to 2018-10-13 18UTC.</b></p>
<p><b>2) Modifiy the notebook to use the MSLP and geopotential ERA5 data that are stored in the Leslie2 folder (6-hourly values over the 20N-45N 60W-0W domain and from october 10th to october 13th).</b></p>
<p><b>2) Compute the CPS diagramm for Leslie.</b></p>
</div>